In [12]:
import torch.nn.functional as F
import urllib
from newspaper import Article
import newspaper
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer, SentimentIntensityAnalyzer
from nltk.sentiment.util import *
import nltk
from torch.utils.data import Dataset, DataLoader
import torch
import os
import re
import pickle

import keras_preprocessing
from keras_preprocessing import sequence
from keras_preprocessing.text import Tokenizer
# from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn import functional as f
import pandas as pd
import json
import numpy as np
import os

DIRECTORY = 'data'
paths = []

for root, dirs, files in os.walk(DIRECTORY):
    for name in files:
        paths.append(os.path.join(root, name))

names = [i.split('/')[-1] for i in paths][1:]
data_dict = dict(zip([i[:-4] for i in names], paths[1:]))

class PreprocessingDataset(Dataset):
    def __init__(self, file, root, x_col, y_col, meta_columns, label_idx = -1):
        self.x_col = x_col
        self.y_col = y_col
        self.data = pd.read_csv(file)
        self.data = self.data.sample(frac=1).reset_index(drop=True)
        self.data = self.data.drop(meta_columns, axis=1)

        # self.data, self.base_ref = self.tokenizer(self.data, [x_col])
        self.x_data = self.data[x_col]
        self.max_len = max([len(i) for i in self.x_data])
        self.max_len = 600

        self.x_data, self.token = self.word_vector(self.x_data)

        # list_x = self.x_data.tolist()
        # for x, entry in enumerate(list_x):
        #     new_entry = self.format_text(entry)
        #     list_x[x] = new_entry

        
        # train_docs = ' '.join(map(str, list_x))

        


        self.data[x_col] = [torch.tensor(i) for i in self.x_data]
        self.data = self.vectorize(self.data, [y_col])
        self.df_data = self.data
        self.data = self.data.to_numpy()

        self.root = root

    def format_text(self, token):
        clean_token = ''.join(chr for chr in token if chr.isalnum() and chr.isalpha())
        return clean_token

    def word_vector(self, data):
        x_data = data
        x_data = list(x_data)
        maximum_length = 0
        max_idx = 0
        for idx, i in enumerate(x_data):

            if len(i) > maximum_length:
                maximum_length = len(i)
                max_idx = idx
        maximum_length = 600
        t = Tokenizer(num_words=600, filters='\n.,:!"#$()&@%^()-_`~[];.,{|}')
        t.fit_on_texts(x_data)
        sequences = t.texts_to_sequences(x_data)
        sequences = sequence.pad_sequences(sequences, maxlen=maximum_length)

        return sequences, t


    def vectorize(self, data_inp, columns):
        data = data_inp
        for column in columns:
            labels = list(data[column].unique())
            ref = dict(zip(data[column].unique(), [i for i in range(len(labels))]))
            data.loc[:, column] = [torch.tensor(ref[data.loc[idx, column]]) for idx in range(len(data))]

        return data

    def __len__ (self):
        return len(self.data)

    def __getitem__ (self, idx):
        
        self.transpose_data = self.data
        self.transpose_data = self.transpose_data.transpose()
        x_data = self.transpose_data[0]
        y_data = self.transpose_data[1]

        return x_data[idx], y_data[idx]

clean_truth_data = PreprocessingDataset(data_dict['politifact_clean_binarized'], DIRECTORY, 'statement', 'veracity', ['source', 'link'])
token_basis = clean_truth_data.token

In [16]:
# sys.getsizeof(clean_truth_data)

torch.Size([])

In [4]:
BATCH_SIZE = 64

primary_data = clean_truth_data #secondary option of truth_data

train_len = int(len(primary_data)*0.8)
test_len = len(primary_data) - train_len

train_set, test_set = torch.utils.data.random_split(primary_data, [train_len, test_len])

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

a = iter(train_loader)
b = np.array(next(a)[0])
inp_size = (b.shape)[1]
emb_dim = 6712800

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class FeedForward(nn.Module):
    def __init__(self, num_classes, input_size, kernel_size=4):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(input_size, 200)
        self.fc3 = nn.Linear(200, 100)        
        self.fc4 = nn.Linear(100, 100)
        self.fc5 = nn.Linear(100, 50)
        self.fc6 = nn.Linear(50, 20)
        self.fc7 = nn.Linear(20, 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.dropout(F.relu(self.fc4(x)))
        x = self.dropout(F.relu(self.fc5(x)))
        x = self.dropout(F.relu(self.fc6(x)))
        x = self.dropout(F.relu(self.fc7(x)))

        
        return x


class RecurrentClassifier(nn.Module):
    def __init__(self, embedding_dim, input_size, hidden_size, output_size, num_layers, dropout=0.3):
        super(RecurrentClassifier, self).__init__()

        self.embedding = nn.Embedding(embedding_dim, input_size)
        self.rnn = nn.LSTM(input_size, 
                            hidden_size,
                            num_layers,
                            batch_first = True,
                            dropout=dropout)
        self.fc1 = nn.Linear(hidden_size*2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)
        x, (hidden, cell) = self.rnn(x)
        print(hidden.shape)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1, :, :]), dim=1))
        x = self.fc1(hidden)
        x = self.dropout(self.fc2(x))

        return x

max_len = len(train_set[1][0])
ref_check = 1

In [11]:

feedforward = FeedForward(ref_check, inp_size).to(device)
x_f_s = next(feedforward.parameters())

recurrent = RecurrentClassifier(emb_dim, inp_size, 50, ref_check, 2, dropout=0.2).to(device)
x_r_s = next(recurrent.parameters())

def model_load(net, PATH, name, export=True):
    if export:
        torch.save(net.state_dict(), PATH+name+'.pth')
        return PATH+name+'.pth'
    else:
        net.torch.load_state_dict(torch.load(PATH + name + '.pth'))
        net.eval()
        return net

model_load(feedforward, 'model_parameters/', 'linear_politifact')
model_load(recurrent, 'model_parameters/', 'lstm_politifact')

'model_parameters/lstm_politifact.pth'

In [17]:
# torch.onnx.export(feedforward, x_f_s, "feedforward.onnx")
# torch.onnx.export(recurrent, x_r_s, "recurrent.onnx")

In [19]:

def meta_extract(url):
    article = Article(url)
    article.download()
    article.parse()
    article.download('punkt')
    article.nlp()
    return article.authors, article.publish_date, article.top_image, article.images, article.title, article.summary

meta_extract('https://www.theatlantic.com/ideas/archive/2022/02/canada-trucker-protests-spread-america/622039/')

(['David Frum', 'About The Author'],
 datetime.datetime(2022, 2, 11, 19, 1, 30, tzinfo=tzutc()),
 'https://cdn.theatlantic.com/thumbor/vbgNQbXpLEtiIr-LOgcZgOtMSaw=/0x133:6230x3378/1200x625/media/img/mt/2022/02/GettyImages_1369598771/original.jpg',
 {"data:image/svg+xml,%3Csvg xmlns='http://www.w3.org/2000/svg' viewBox='0 0 266 200' /%3E",
  'https://cdn.theatlantic.com/thumbor/bcKXdkaXewFxIHnPdLEdYytYgYE=/0x0:6240x3510/960x540/media/img/mt/2022/02/GettyImages_1369598771/original.jpg',
  'https://cdn.theatlantic.com/thumbor/vbgNQbXpLEtiIr-LOgcZgOtMSaw=/0x133:6230x3378/1200x625/media/img/mt/2022/02/GettyImages_1369598771/original.jpg'},
 'Canada’s Trucker Blockades Are a Warning',
 'Much of the money donated in support of the Canadian protests has been raised internationally, especially in the United States.\nThat’s the busiest of all the border crossings between the United States and Canada, crucial to moving auto parts.\nAbout 90 percent of Canadian truck drivers are vaccinated; compar